In [1]:
from ift6758.data.acquisition import NHLGameData

In [2]:
data_path_raw = './../ift6758/data/json_raw/'
nhl_games_data = NHLGameData(data_path_raw)

In [3]:
for year in range(2016,2021):
    nhl_games_data.fetch_season(year)

Loading from cache file ./../ift6758/data/json_raw/2016\2016-regular.pkl
Found 1230 regular games for season 2016-2017
Loading from cache file ./../ift6758/data/json_raw/2016\2016-playoff.pkl
Found 102 playoff games for season 2016-2017
Loading from cache file ./../ift6758/data/json_raw/2017\2017-regular.pkl
Found 1271 regular games for season 2017-2018
Loading from cache file ./../ift6758/data/json_raw/2017\2017-playoff.pkl
Found 105 playoff games for season 2017-2018
Loading from cache file ./../ift6758/data/json_raw/2018\2018-regular.pkl
Found 1271 regular games for season 2018-2019
Loading from cache file ./../ift6758/data/json_raw/2018\2018-playoff.pkl
Found 105 playoff games for season 2018-2019
Loading from cache file ./../ift6758/data/json_raw/2019\2019-regular.pkl
Found 1271 regular games for season 2019-2020
Loading from cache file ./../ift6758/data/json_raw/2019\2019-playoff.pkl
Found 105 playoff games for season 2019-2020
Loading from cache file ./../ift6758/data/json_raw/2

In [4]:
from ift6758.data.cleaning import DataCleaner

In [5]:
data_path_clean = './../ift6758/data/json_clean/'
data_cleaner = DataCleaner(data_raw=nhl_games_data, data_path_clean=data_path_clean)

In [8]:
for year in range(2016,2021):
    data_cleaner.clean_season(year)

### Test extract

In [10]:
import pandas as pd

# print out a pandas head of the data
data = pd.read_pickle(data_path_clean + '2016/2016.pkl')
    
data.head(10)

,game_id,time,event_type,period,team,coordinates,shooter,goalie,shot_type,empty_net,strength
0,2016020001,01:11,SHOT,1,Toronto Maple Leafs,"{'x': -77.0, 'y': 5.0}",Mitchell Marner,Craig Anderson,Wrist Shot,False,None
1,2016020001,02:53,SHOT,1,Ottawa Senators,"{'x': 86.0, 'y': 13.0}",Chris Kelly,Frederik Andersen,Wrist Shot,False,None
2,2016020001,04:01,SHOT,1,Ottawa Senators,"{'x': 23.0, 'y': -38.0}",Cody Ceci,Frederik Andersen,Wrist Shot,False,None
3,2016020001,04:46,SHOT,1,Ottawa Senators,"{'x': 33.0, 'y': -15.0}",Erik Karlsson,Frederik Andersen,Slap Shot,False,None
4,2016020001,06:46,SHOT,1,Toronto Maple Leafs,"{'x': -34.0, 'y': 28.0}",Martin Marincin,Craig Anderson,Wrist Shot,False,None
5,2016020001,07:30,SHOT,1,Toronto Maple Leafs,"{'x': -33.0, 'y': -17.0}",Mitchell Marner,Craig Anderson,Wrist Shot,False,None
6,2016020001,08:21,GOAL,1,Toronto Maple Leafs,"{'x': -70.0, 'y': 1.0}",Auston Matthews,Craig Anderson,Wrist Shot,False,"{'code': 'EVEN', 'name': 'Even'}"
7,2016020001,08:29,SHOT,1,Toronto Maple Leafs,"{'x': -45.0, 'y': -36.0}",Matt Martin,Craig Anderson,Wrist Shot,False,None
8,2016020001,09:00,SHOT,1,Ottawa Senators,"{'x': 33.0, 'y': -18.0}",Erik Karlsson,Frederik Andersen,Slap Shot,False,None
9,2016020001,10:16,SHOT,1,Ottawa Senators,"{'x': 34.0, 'y': 20.0}",Erik Karlsson,Frederik Andersen,Wrist Shot,False,None
